https://github.com/wangshier108/ComfyUI_RAGDemo/blob/main/test_rag.py

In [1]:
!pip install langchain langchain-community langchain-milvus pymilvus milvus sentence-transformers torch transformers accelerate PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [4]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 4.7 MB/s eta 0:00:00


In [2]:
!python 1.py

/content/1.py:8: LangChainDeprecationWarning: Importing HuggingFaceBgeEmbeddings from langchain.embeddings is deprecated. Please replace deprecated imports:

>> from langchain.embeddings import HuggingFaceBgeEmbeddings

with new imports of:

>> from langchain_community.embeddings import HuggingFaceBgeEmbeddings
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.embeddings import HuggingFaceBgeEmbeddings
usage: 1.py [-h] [--pdf_path PDF_PATH] [--llm_path LLM_PATH]
            [--device {cpu,cuda}]
            question
1.py: error: the following arguments are required: question


In [ ]:
!python 1.py "What is the relationship between Harry and Halley?"

In [7]:
!huggingface-cli login --token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `read` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `read`


In [ ]:
/content/1.py





import os
import argparse # لاستقبال المدخلات من سطر الأوامر

# --- استيراد المكتبات الأساسية (نفس ما في الكود الأصلي) ---
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_milvus.vectorstores import Milvus
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# --- استيراد LLM المخصص ---
# افترض أن test_llm.py موجود في نفس المجلد
from test_llm import LLaMA3_1_LLM

# --- دالة الـ RAG الرئيسية (مأخوذة من دالة test الأصلية مع تعديلات) ---
def perform_rag(question: str, pdf_file_path: str, llm_model_path: str, embedding_device: str = 'cpu'):
    print(f"السؤال: {question}")
    print(f"مسار ملف PDF: {pdf_file_path}")
    print(f"مسار نموذج LLM: {llm_model_path}")
    print(f"جهاز التضمين: {embedding_device}")

    if not os.path.exists(pdf_file_path):
        print(f"خطأ: ملف PDF غير موجود في المسار: {pdf_file_path}")
        return "خطأ: ملف PDF غير موجود."

    # 1. تحميل ومعالجة PDF
    print("جارٍ تحميل ومعالجة ملف PDF...")
    loader = PyPDFLoader(file_path=pdf_file_path)
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)
    print(f"تم تقسيم المستند إلى {len(docs)} جزء.")

    # 2. إعداد نموذج التضمين
    print("جارٍ إعداد نموذج التضمين...")
    model_name_embedding = "BAAI/bge-base-zh-v1.5"
    model_kwargs_embedding = {'device': embedding_device}
    encode_kwargs_embedding = {'normalize_embeddings': True}
    embedding_model = HuggingFaceBgeEmbeddings(
        model_name=model_name_embedding,
        model_kwargs=model_kwargs_embedding,
        encode_kwargs=encode_kwargs_embedding,
        query_instruction="为这个句子生成表示以用于检索相关文章："
    )
    print("تم إعداد نموذج التضمين.")

    # 3. إعداد قاعدة البيانات المتجهة (Milvus)
    print("جارٍ إعداد قاعدة البيانات المتجهة Milvus...")
    # تأكد من أن لديك صلاحيات للكتابة في المجلد الحالي لإنشاء milvus_demo.db
    # أو قم بتكوين Milvus للاتصال بخادم إذا كان لديك واحد.
    try:
        vectorstore = Milvus.from_documents(
            documents=docs,
            embedding=embedding_model,
            collection_name="standalone_book_collection", # اسم مجموعة مختلف لتجنب التعارض
            drop_old=True, # احذر: هذا سيحذف المجموعة القديمة بنفس الاسم
            connection_args={"uri": "./milvus_standalone_demo.db"}, # ملف قاعدة بيانات مختلف
        )
        print("تم إعداد Milvus بنجاح.")
    except Exception as e:
        print(f"خطأ أثناء إعداد Milvus: {e}")
        print("تأكد من تثبيت مكتبات Milvus (pymilvus, milvus) بشكل صحيح.")
        print("إذا كنت تستخدم Milvus server، تأكد أنه يعمل ويمكن الوصول إليه.")
        return "خطأ في إعداد Milvus."

    # 4. إعداد الـ Prompt و الـ Retriever
    PROMPT_TEMPLATE = """
    Human: You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.
    Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    <context>
    {context}
    </context>

    <question>
    {question}
    </question>

    The response should be specific and use statistics or numbers when possible.

    Assistant:"""
    prompt_template = PromptTemplate(
        template=PROMPT_TEMPLATE, input_variables=["context", "question"]
    )
    retriever = vectorstore.as_retriever()

    def format_docs(docs_list):
        return "\n\n".join(doc.page_content for doc in docs_list)

    # 5. إعداد LLM
    print(f"جارٍ تحميل نموذج LLM من: {llm_model_path}...")
    try:
        # افترض أن LLaMA3_1_LLM في test_llm.py يمكنه التعامل مع هذا المسار
        llm = LLaMA3_1_LLM(mode_name_or_path=llm_model_path)
        print("تم تحميل نموذج LLM.")
    except Exception as e:
        print(f"خطأ أثناء تحميل نموذج LLM: {e}")
        print("تأكد أن المسار صحيح وأن ملف test_llm.py يمكنه تحميل النموذج من هذا المسار.")
        print("قد تحتاج إلى تعديل كود test_llm.py أو توفير مسار محلي للنموذج إذا كان يتوقع ذلك.")
        return "خطأ في تحميل LLM."


    # 6. بناء وتشغيل سلسلة RAG
    print("جارٍ بناء وتشغيل سلسلة RAG...")
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt_template
        | llm
        | StrOutputParser()
    )

    try:
        response = rag_chain.invoke(question)
        print("\n--- الإجابة ---")
        print(response)
        print("--- --- ---")
        return response
    except Exception as e:
        print(f"خطأ أثناء تشغيل سلسلة RAG: {e}")
        return "خطأ أثناء توليد الإجابة."

# --- نقطة الدخول الرئيسية للسكربت ---
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="قم بتشغيل RAG على ملف PDF باستخدام LLM.")
    parser.add_argument("question", type=str, help="السؤال الذي تريد طرحه.")
    parser.add_argument("--pdf_path", type=str, default="test.pdf", help="المسار إلى ملف PDF (الافتراضي: test.pdf في نفس المجلد).")
    parser.add_argument("--llm_path", type=str, default="meta-llama/Llama-3.2-1B-Instruct", help="مسار أو اسم نموذج LLM (الافتراضي: meta-llama/Llama-3.2-1B-Instruct).")
    parser.add_argument("--device", type=str, default="cpu", choices=['cpu', 'cuda'], help="الجهاز المستخدم لنموذج التضمين (cpu أو cuda).")

    args = parser.parse_args()

    # --- تأكد من وجود ملف test_llm.py ---
    if not os.path.exists("test_llm.py"):
        print("خطأ: ملف test_llm.py غير موجود في نفس المجلد.")
        print("هذا الملف ضروري لتحميل النموذج اللغوي الكبير.")
        exit()

    # --- تشغيل دالة RAG ---
    perform_rag(args.question, args.pdf_path, args.llm_path, args.device)

In [ ]:
/content/test_llm.py




from langchain.llms.base import LLM
from typing import Any, List, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

class LLaMA3_1_LLM(LLM):
    # 基于本地 llama3.1 自定义 LLM 类
    tokenizer: AutoTokenizer = None
    model: AutoModelForCausalLM = None

    def __init__(self, mode_name_or_path :str):

        super().__init__()
        print("正在从本地加载模型...")
        self.tokenizer = AutoTokenizer.from_pretrained(mode_name_or_path, use_fast=False)
        self.model = AutoModelForCausalLM.from_pretrained(mode_name_or_path, torch_dtype=torch.float32, device_map="auto")
        self.tokenizer.pad_token = self.tokenizer.eos_token
        print("完成本地模型的加载")


    def _call(self, prompt : str, stop: Optional[List[str]] = None,
                run_manager: Optional[CallbackManagerForLLMRun] = None,
                **kwargs: Any):
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]

        input_ids = self.tokenizer.apply_chat_template(messages,tokenize=False,add_generation_prompt=True)
        model_inputs = self.tokenizer([input_ids], return_tensors="pt").to(self.model.device)
        generated_ids = self.model.generate(model_inputs.input_ids,max_new_tokens=512)
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        return response

    @property
    def _llm_type(self) -> str:
        return "LLaMA3_1_LLM"

In [8]:
!python 1.py "What is the relationship between Harry and Halley?" --llm_path "meta-llama/Llama-3.2-1B-Instruct"

/content/1.py:8: LangChainDeprecationWarning: Importing HuggingFaceBgeEmbeddings from langchain.embeddings is deprecated. Please replace deprecated imports:

>> from langchain.embeddings import HuggingFaceBgeEmbeddings

with new imports of:

>> from langchain_community.embeddings import HuggingFaceBgeEmbeddings
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.embeddings import HuggingFaceBgeEmbeddings
السؤال: What is the relationship between Harry and Halley?
مسار ملف PDF: test.pdf
مسار نموذج LLM: meta-llama/Llama-3.2-1B-Instruct
جهاز التضمين: cpu
جارٍ تحميل ومعالجة ملف PDF...
تم تقسيم المستند إلى 2 جزء.
جارٍ إعداد نموذج التضمين...
/content/1.py:41: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and sho

In [ ]:
/content/1.py:8: LangChainDeprecationWarning: Importing HuggingFaceBgeEmbeddings from langchain.embeddings is deprecated. Please replace deprecated imports:

>> from langchain.embeddings import HuggingFaceBgeEmbeddings

with new imports of:

>> from langchain_community.embeddings import HuggingFaceBgeEmbeddings
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.embeddings import HuggingFaceBgeEmbeddings
السؤال: What is the relationship between Harry and Halley?
مسار ملف PDF: test.pdf
مسار نموذج LLM: meta-llama/Llama-3.2-1B-Instruct
جهاز التضمين: cpu
جارٍ تحميل ومعالجة ملف PDF...
تم تقسيم المستند إلى 2 جزء.
جارٍ إعداد نموذج التضمين...
/content/1.py:41: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceBgeEmbeddings(
2025-05-30 23:31:20.146392: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
E0000 00:00:1748647880.185808    4320 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748647880.197919    4320 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-30 23:31:20.236448: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
تم إعداد نموذج التضمين.
جارٍ إعداد قاعدة البيانات المتجهة Milvus...
2025-05-30 23:31:33,025 [DEBUG][_create_connection]: Created new connection using: 40147fddcdfa47e3804cfa7f0155cf1e (async_milvus_client.py:599)
تم إعداد Milvus بنجاح.
جارٍ تحميل نموذج LLM من: meta-llama/Llama-3.2-1B-Instruct...
正在从本地加载模型...
tokenizer_config.json: 100% 54.5k/54.5k [00:00<00:00, 5.94MB/s]
tokenizer.json: 100% 9.09M/9.09M [00:00<00:00, 48.8MB/s]
special_tokens_map.json: 100% 296/296 [00:00<00:00, 1.53MB/s]
config.json: 100% 877/877 [00:00<00:00, 3.84MB/s]
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
WARNING:huggingface_hub.file_download:Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
model.safetensors: 100% 2.47G/2.47G [00:34<00:00, 71.4MB/s]
generation_config.json: 100% 189/189 [00:00<00:00, 1.08MB/s]
完成本地模型的加载
تم تحميل نموذج LLM.
جارٍ بناء وتشغيل سلسلة RAG...
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.

--- الإجابة ---
根据提供的描述，Harry (哈利) 和 Halley (哈雷) 的关系可以说是亲密的朋友和伴侣。虽然没有提供明确的证据表明他们是夫妻关系，但根据他们的行为和共同的经历，可以推断出他们是 close 的朋友。

在村庄中，他们一起经历了多次危险情况，但他们也共同成为了村民们的hero。他们的故事在村庄传为佳话，并且在夜幕降临时，哈雷总是骑上哈利，确保一切安宁。
--- --- ---
E20250530 23:34:23.003249  4471 server.cpp:47] [SERVER][BlockLock][milvus] Process exit
Error in sys.excepthook:

Original exception was:

In [9]:
!python 1.py "What are the effects of climate change?" --llm_path "meta-llama/Llama-3.2-1B-Instruct"

/content/1.py:8: LangChainDeprecationWarning: Importing HuggingFaceBgeEmbeddings from langchain.embeddings is deprecated. Please replace deprecated imports:

>> from langchain.embeddings import HuggingFaceBgeEmbeddings

with new imports of:

>> from langchain_community.embeddings import HuggingFaceBgeEmbeddings
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.embeddings import HuggingFaceBgeEmbeddings
السؤال: What are the effects of climate change?
مسار ملف PDF: test.pdf
مسار نموذج LLM: meta-llama/Llama-3.2-1B-Instruct
جهاز التضمين: cpu
خطأ: ملف PDF غير موجود في المسار: test.pdf


In [ ]:
!!python rag_standalone_en.py "Your question?" --pdf_path "your_english_book.pdf" --llm_path "meta-llama/Llama-3.2-1B-Instruct" --embedding_model "sentence-transformers/all-MiniLM-L6-v2"

In [ ]:
/content/Understanding_Climate_Change.pdf

In [ ]:
/content/2.py




import os
import argparse

# --- Core library imports ---
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_milvus.vectorstores import Milvus

# Updated import for HuggingFaceBgeEmbeddings
# Option 1 (current common practice for langchain_community):
from langchain_community.embeddings import HuggingFaceEmbeddings as CommunityHuggingFaceEmbeddings
# Option 2 (newer, if you install langchain-huggingface):
# from langchain_huggingface import HuggingFaceEmbeddings

from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# --- Custom LLM import ---
# Assuming test_llm.py is in the same directory
from test_llm import LLaMA3_1_LLM

# --- Main RAG function ---
def perform_rag(question: str, pdf_file_path: str, llm_model_path: str, embedding_model_name: str, embedding_device: str = 'cpu'):
    print(f"Question: {question}")
    print(f"PDF file path: {pdf_file_path}")
    print(f"LLM model path/name: {llm_model_path}")
    print(f"Embedding model name: {embedding_model_name}")
    print(f"Embedding device: {embedding_device}")

    if not os.path.exists(pdf_file_path):
        print(f"Error: PDF file not found at: {pdf_file_path}")
        return "Error: PDF file not found."

    # 1. Load and process PDF
    print("Loading and processing PDF file...")
    loader = PyPDFLoader(file_path=pdf_file_path)
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)
    print(f"Document split into {len(docs)} chunks.")

    # 2. Setup embedding model
    print("Setting up embedding model...")
    # Using CommunityHuggingFaceEmbeddings from langchain_community
    # If you installed `langchain-huggingface`, you could use `HuggingFaceEmbeddings` from there.
    embedding_model = CommunityHuggingFaceEmbeddings(
        model_name=embedding_model_name,
        model_kwargs={'device': embedding_device},
        encode_kwargs={'normalize_embeddings': True} # Important for cosine similarity
    )
    # If using BAAI/bge models, they might have a 'query_instruction'
    # For 'sentence-transformers/all-MiniLM-L6-v2', it's not typically needed.
    # If you switch back to a BGE model, you might add:
    # query_instruction="Represent this sentence for searching relevant passages:"
    print("Embedding model setup complete.")

    # 3. Setup vector store (Milvus)
    print("Setting up Milvus vector store...")
    try:
        vectorstore = Milvus.from_documents(
            documents=docs,
            embedding=embedding_model,
            collection_name="rag_standalone_collection_en", # Different collection name
            drop_old=True,
            connection_args={"uri": "./milvus_standalone_en.db"}, # Different DB file
        )
        print("Milvus setup successful.")
    except Exception as e:
        print(f"Error during Milvus setup: {e}")
        print("Ensure Milvus libraries (pymilvus, milvus) are correctly installed.")
        print("If using a Milvus server, ensure it's running and accessible.")
        return "Error setting up Milvus."

    # 4. Setup Prompt and Retriever
    PROMPT_TEMPLATE_EN = """
    Human: You are an AI assistant. Provide concise answers to questions using the
    following pieces of context. If you don't know the answer, just say that you don't know.
    Do not try to make up an answer.

    Context:
    {context}

    Question:
    {question}

    Assistant:"""
    prompt_template = PromptTemplate(
        template=PROMPT_TEMPLATE_EN, input_variables=["context", "question"]
    )
    retriever = vectorstore.as_retriever()

    def format_docs_for_rag(docs_list):
        return "\n\n".join(doc.page_content for doc in docs_list)

    # 5. Setup LLM
    print(f"Loading LLM from: {llm_model_path}...")
    try:
        llm = LLaMA3_1_LLM(mode_name_or_path=llm_model_path) # Assumes LLaMA3_1_LLM in test_llm.py handles this
        print("LLM loaded.")
    except Exception as e:
        print(f"Error loading LLM: {e}")
        print("Ensure the path is correct and test_llm.py can load the model from this path/name.")
        print("You might need to adjust test_llm.py or provide a local model path if it expects one.")
        return "Error loading LLM."

    # 6. Build and run RAG chain
    print("Building and running RAG chain...")
    rag_chain = (
        {"context": retriever | format_docs_for_rag, "question": RunnablePassthrough()}
        | prompt_template
        | llm
        | StrOutputParser()
    )

    try:
        response = rag_chain.invoke(question)
        print("\n--- Answer ---")
        print(response)
        print("--- --- ---")
        return response
    except Exception as e:
        print(f"Error during RAG chain execution: {e}")
        return "Error generating answer."

# --- Main script entry point ---
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Run RAG on a PDF file using an LLM.")
    parser.add_argument("question", type=str, help="The question to ask.")
    parser.add_argument("--pdf_path", type=str, default="english_book.pdf", help="Path to the PDF file (default: english_book.pdf in the same directory).")
    parser.add_argument("--llm_path", type=str, default="meta-llama/Llama-3.2-1B-Instruct", help="Path or Hugging Face name of the LLM (default: meta-llama/Llama-3.2-1B-Instruct).")
    parser.add_argument("--embedding_model", type=str, default="sentence-transformers/all-MiniLM-L6-v2", help="Hugging Face name of the embedding model (default: sentence-transformers/all-MiniLM-L6-v2).")
    parser.add_argument("--device", type=str, default="cpu", choices=['cpu', 'cuda'], help="Device for the embedding model (cpu or cuda).")

    args = parser.parse_args()

    if not os.path.exists("test_llm.py"):
        print("Error: test_llm.py not found in the current directory.")
        print("This file is required to load the Large Language Model.")
        exit()

    perform_rag(args.question, args.pdf_path, args.llm_path, args.embedding_model, args.device)

In [2]:
!python 2.py "What are the effects of climate change?" --pdf_path "Understanding_Climate_Change.pdf" --llm_path "meta-llama/Llama-3.2-1B-Instruct" --embedding_model "sentence-transformers/all-MiniLM-L6-v2"

Question: What are the effects of climate change?
PDF file path: Understanding_Climate_Change.pdf
LLM model path/name: meta-llama/Llama-3.2-1B-Instruct
Embedding model name: sentence-transformers/all-MiniLM-L6-v2
Embedding device: cpu
Loading and processing PDF file...
Document split into 33 chunks.
Setting up embedding model...
/content/2.py:47: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = CommunityHuggingFaceEmbeddings(
2025-05-30 23:47:44.731334: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 